In [144]:
import pandas as pd
import json
import collections
import scipy.io
import numpy as np
import mpu
import math

In [289]:
bus = scipy.io.loadmat('MATPOWER/bus.mat')['bus']
power_demand = bus[:, 2]
df = pd.read_csv("GIS/CATS_gens.csv")
df = df[df['Pmax'] != 0.0].to_numpy()
#df = df.drop_duplicates(subset=['PlantCode','GenID']).to_numpy()

In [368]:
gen_emission = pd.read_excel("emissions2022.xlsx")
gen_emission = gen_emission[gen_emission['Generation (kWh)'] > 0]
gen_emission = gen_emission[gen_emission['State'] == 'CA']
gen_emission = gen_emission[['Plant Code','Fuel Code','Aggregated Fuel Group','carbon emission rate']]
gen_emission = gen_emission.rename(columns={'Plant Code': 'Plant_Code'})
gen_emission.groupby(['Fuel Code'])['carbon emission rate'].mean()

Fuel Code
BIT    4.702783
DFO    1.191170
GEO    0.028590
JF     0.988836
MSW    1.004947
NG     0.603798
PG     1.331544
WO     0.400283
Name: carbon emission rate, dtype: float64

In [246]:
type_to_emission = collections.defaultdict(float)
type_to_emission['Conventional Hydroelectric'] = 0.001
type_to_emission['Hydroelectric Pumped Storage'] = 0.001
type_to_emission['Petroleum Liquids'] = 1.191
type_to_emission['Natural Gas Internal Combustion Engine'] = 0.604
type_to_emission['Natural Gas Fired Combined Cycle'] = 0.604
type_to_emission['Natural Gas Steam Turbine'] = 0.604
type_to_emission['Natural Gas Fired Combustion Turbine'] = 0.604
#type_to_emission['Nuclear'] = 0
type_to_emission['Geothermal'] = 0.029
#type_to_emission['Onshore Wind Turbine'] = 0
type_to_emission['Other Waste Biomass'] = 0.1
type_to_emission['Wood/Wood Waste Biomass'] = 0.1
type_to_emission['Landfill Gas'] = 0.604
type_to_emission['Solar Photovoltaic'] = 0.001
type_to_emission['Solar Thermal without Energy Storage'] = 0.001
type_to_emission['Conventional Steam Coal'] = 4.703
type_to_emission['Other Gases'] = 0.8
type_to_emission['Batteries'] = 0.001
#type_to_emission['Petroleum Coke'] = 1808.5
type_to_emission['Municipal Solid Waste'] = 1.005
type_to_emission['Other Natural Gas'] = 0.604
type_to_emission['All Other'] = 0.1
#type_to_emission['IMPORT'] = 884
#type_to_emission['Synchronous Condenser'] = 884

In [290]:
branch_ = scipy.io.loadmat('MATPOWER/branch2.mat')['B']
branch_from_bus = list(map(int, branch_[:, 0]-1))
branch_to_bus = list(map(int, branch_[:, 1]-1))
line_to_nodes = [list( map(int,i) ) for i in branch_[:, 0:2]-1]


In [291]:
f = open("pf_solution.json")
sol = json.load(f)
gen = [0]*2149
gen_cost = [0]*2149
power_generation = [0]*2149
carbon_emission = [0]*2149
branch_power_to = [0]*10574
branch_power_from = [0]*10574
for line, val in sol['solution']['gen'].items():
    if val['pg'] != 0.0:
        gen[int(line)-1] = df[int(line)-1][2]-1
        carbon_emission[int(line)-1] = type_to_emission[df[int(line)-1][3]]
        gen_cost[int(line)-1] = val['pg_cost']*100
        power_generation[int(line)-1] = val['pg']*100

for line, val in sol['solution']['branch'].items():
    branch_power_from[int(line)-1] = val['pf']*100
    branch_power_to[int(line)-1] = val['pt']*100

f.close()

In [292]:
graph = collections.defaultdict(list) # from: (to, line)
graph_reverse = collections.defaultdict(list) # to: (from, line)
for i, (from_bus, to_bus) in enumerate(line_to_nodes):
    graph[from_bus].append((to_bus, i))
    graph_reverse[to_bus].append((from_bus, i))
for i, f in enumerate(branch_power_from):
    if f < 0.0:
        from_node, to_node = line_to_nodes[i]
        graph[from_node].remove((to_node, i))
        graph[to_node].append((from_node, i))
        graph_reverse[to_node].remove((from_node, i))
        graph_reverse[from_node].append((to_node, i))
        branch_power_from[i] = -f

In [294]:
num_bus = len(bus)
num_branch = len(branch_from_bus)
num_gen = len(gen)
print("Number of transmission lines and transformers: " + str(num_branch))
print("Number of buses: " + str(num_bus))
print("Number of generators: " + str(num_gen))

Number of transmission lines and transformers: 10574
Number of buses: 8870
Number of generators: 2149


In [317]:
# Tarjan's Strongly Connected Component (SCC) Algorithm
UNVISITED = -1
id = [0]
sccCount = [0]
ids = [0]*num_bus
low = [0]*num_bus
onStack = [False]*num_bus
stack = []
def findSccs():
    for i in range(num_bus): ids[i] = UNVISITED
    for i in range(num_bus):
        if ids[i] == UNVISITED:
            tarjan_dfs(i)
    return low
def tarjan_dfs(at):
    stack.append(at)
    onStack[at] = True
    ids[at] = id[0]
    low[at] = id[0]
    id[0] += 1
    for nei, _ in graph[at]:
        if ids[nei] == UNVISITED:
            tarjan_dfs(nei)
        if onStack[nei]:
            low[at] = min(low[nei], low[at])
    if ids[at] == low[at]:
        while stack:
            node = stack.pop(-1)
            onStack[node] = False
            low[node] = ids[at]
            if node == at: break
        sccCount[0] += 1

In [318]:
low = findSccs()

In [325]:
low[0]

0

In [319]:
sccCount[0]

8857

In [323]:
graph[0]

[]

In [309]:
# Kahn's algo
def topo_order_kahn(graph):
    in_degree = [0]*num_bus
    for i in range(num_bus):
        for nei, _ in graph[i]:
            in_degree[nei] += 1
    q = []
    for i in range(num_bus):
        if in_degree[i] == 0:
            q.append(i)
    index = 0
    order = [0]*num_bus
    while q:
        at = q.pop(0)
        order[index] = at
        index += 1
        for nei, _ in graph[at]:
            in_degree[nei] -= 1
            if in_degree[nei] == 0:
                q.append(nei)
    return in_degree

In [310]:
in_degree = topo_order_kahn(graph)

In [321]:
res = 0
for i, v in enumerate(in_degree):
    if v != 0:
        print(i)

0
1
2
3
4
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
36
40
42
43
45
47
48
49
51
53
69
71
72
73
76
77
78
79
80
81
83
85
87
88
90
91
93
94
95
96
98
99
101
102
105
166
167
173
178
326
327
328
331
344
369
445
478
480
571
577
578
579
580
581
582
583
584
585
586
589
590
591
592
593
594
595
596
597
598
599
608
609
613
616
619
621
622
624
625
626
635
636
638
639
641
642
644
645
646
647
648
650
651
655
656
657
658
659
660
661
662
663
666
667
668
669
671
672
676
677
678
682
684
686
687
688
689
690
692
693
694
696
698
700
705
709
710
712
714
715
717
719
720
721
722
723
725
726
728
729
734
735
736
737
738
739
740
741
742
743
744
745
746
747
748
749
750
751
752
753
754
755
756
757
758
759
760
817
818
819
820
822
823
824
827
828
829
830
831
833
834
835
836
837
838
839
840
841
842
843
844
846
847
849
850
851
852
854
855
856
857
859
860
861
862
863
864
865
866
867
868
869
870
871
872
873
874
875
877
878
880
881
882
883
884
885
886
887
888
890
891
893
894
895
896
897
898
899
900
901
902
903
906


In [250]:
line_to_gen = collections.defaultdict(set)
node_to_gen = collections.defaultdict(set)
def dfs(g, n, visited):
    if n in visited: return
    visited.add(n)
    node_to_gen[n].add(g)
    for nei, line in graph[n]:
        line_to_gen[line].add(g)
        dfs(g, nei, visited)
for i, g in enumerate(gen):
    visited = set()
    dfs(g, g, visited)
line_prop_mat=np.zeros((num_gen, num_branch), dtype=float)
bus_prop_mat=np.zeros((num_gen, num_bus), dtype=float)
# initially, if node_to_gen only has 1 generator, bus_prop_mat start with 1
for k, v in node_to_gen.items():
    if len(v) == 1:
        idx = list(gen).index(list(v)[0])
        bus_prop_mat[idx][int(k)] = 1.0
for k, v in line_to_gen.items():
    if len(v) == 1:
        idx = list(gen).index(list(v)[0])
        line_prop_mat[idx][int(k)] = 1.0
# visit nodes in topological order
# step 1: calculate bus_prop based on in flowing lines_prop
# step 2: calculate line_prop of out flowing lines
in_degree = collections.defaultdict(int)
for i, v in graph_reverse.items():
    in_degree[i] = len(v)
q = [] # list of nodes with no inflow
topo_order = []
for g in gen:
    if in_degree[g] == 0:
        q.append(g)
while q:
    cur = int(q.pop(0))
    if len(node_to_gen[cur]) > 1:
        out_total = power_demand[cur]
        for nei, out_line in graph[cur]:
            out_total += branch_power_from[out_line]
        
        for g in node_to_gen[cur]:
            idx = list(gen).index(g)
            if cur == g:
                if out_total > 0.000001: 
                        #print("Power gen: " + str(power_generation[idx]))
                    bus_prop_mat[idx][cur] = power_generation[idx]/out_total
            else:
                for nei, in_line in graph_reverse[cur]:
                    if out_total > 0.000001: 
                        bus_prop_mat[idx][cur] += branch_power_from[in_line]*line_prop_mat[idx][in_line]/out_total
        for g in node_to_gen[cur]:
            idx = list(gen).index(g)
            for nei, out_line in graph[cur]:
                line_prop_mat[idx][out_line] = bus_prop_mat[idx][cur]

    topo_order.append(cur)
    for nei, line in graph[cur]:
        in_degree[nei] -= 1
        if in_degree[nei] == 0:
            q.append(nei)


In [178]:
graph_reverse[15]

[(3960.0, 3145)]

In [251]:
in_degree_list = [0]*num_bus
for k, v in in_degree.items():
    in_degree_list[int(k)] = v
def dfs_cycle(n, visited, total_gen, temp, in_degree_list):
    if n in visited: return
    visited.add(n)
    if n in gen:
        total_gen[0] += power_generation[gen.index(n)]
        temp[n] = power_generation[gen.index(n)]
    for nei, _ in graph[n]:
        if in_degree_list[int(nei)] > 0:
            dfs_cycle(int(nei), visited, total_gen, temp, in_degree_list)
            in_degree_list[int(nei)] -= 1

In [252]:
for i in range(len(in_degree_list)):
    if in_degree_list[i] > 0 and i in gen and power_generation[gen.index(i)] > 0.000001 and len(graph[i]) != 0:
        total_gen = [0.0]
        visited = set()
        temp = collections.defaultdict(float)
        dfs_cycle(i, visited, total_gen, temp, in_degree_list)
        if total_gen != 0:
            for k, v in temp.items():
                print(v/total_gen[0])
                bus_prop_mat[gen.index(k)][k] = v/total_gen[0]


0.5638751875786592
0.4361248124213408
1.0
0.8214283601178055
0.17857163988219446
1.0
1.0
0.33354926072088176
0.6664507392791182
1.0
0.6503279674734895
0.0039414390701801045
0.03567170263828473
0.24962367544226247
0.060435215375783
1.0
1.0
0.20877415114261993
0.79122584885738
1.0
1.0
1.0
0.18181874468456266
0.8181812553154374
0.49999999999999956
0.5000000000000004
1.0
1.0
0.1176471706038275
0.8823528293961724
1.0
0.20694068514969863
0.6688948206537514
0.12416449419655003
1.0
1.0
1.0
0.4323887395372227
0.5676112604627772
0.14701560473868028
0.009461687870556963
0.09114316887944386
0.003659035515234736
0.008601534720791145
0.665278587357655
0.004656953380451375
0.006320149838177376
0.0006652818996266429
0.0069854284163533365
0.0049895926742058744
0.019353448968179896
0.0016631997662832645
0.0033263962157895075
0.005590998624076071
0.0036590355011958133
0.004656953377017083
0.003659035510960397
0.003326396211128069
0.005987510534193548
1.0
1.0
0.0172762696754256
0.037176788541332315
0.0074

In [228]:
res = 0
for i, v in enumerate(in_degree_list):
    if v > 0:
        res += 1
res

69

In [304]:
np.unravel_index(bus_prop_mat.argmax(), bus_prop_mat.shape)

(1182, 490)

In [256]:
res = 0
for i, v in enumerate(sum(bus_prop_mat)):
    if abs(v) < 0.0001:
        res += 1
        #print(str(i) + " :" + str(v))
res

3586

In [247]:
avg_carbon_emission_rate_node = carbon_emission @ bus_prop_mat

In [248]:
df_ = pd.read_csv("GIS/CATS_buses.csv")
df_.insert(2, 'power_demand', power_demand)
df_.insert(3, 'avg carbon emission', avg_carbon_emission_rate_node)
df_.to_csv('GIS/CATS_buses_output.csv', index=False)